<a href="https://colab.research.google.com/github/alitourani/deep-learning-from-scratch/blob/main/Codes/ML/Multi_Label_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple-Label Classification with ANNs**

🎯 **Goal:** We want to classify data into zero or more class labels. It involves predicting the likelihood across two or more class labels. An assumption is that the input does not belong to one class only (labels are not mutually exclusive).

🗄️ **Dataset:** Generated.


**1. Import Libraries**

In [22]:
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from numpy import mean, std, asarray
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold
from sklearn.datasets import make_multilabel_classification

**2. Creating the Dataset**

In [23]:
# Variables
numberOfSamples = 500
numberOfFeatures = 10 # input features
numberOfClasses = 3 # output features, i.e., class label outputs for each sample
numberOfLabels = 2 # values for each class
randomState = 1

# Defining the dataset
features, labels = make_multilabel_classification(n_samples = numberOfSamples,
                                      n_features = numberOfFeatures,
                                      n_classes = numberOfClasses,
                                      n_labels = numberOfLabels,
                                      random_state = randomState)

# Summarize the dataset shape
print(f'Shape of the features is {features.shape}, while the shape of the labes is {labels.shape}.\n')

# Showing some of the results
for item in range(5):
  print(f'Item#{item}: Features={features[item]}, Labels={labels[item]}')

Shape of the features is (500, 10), while the shape of the labes is (500, 3).

Item#0: Features=[ 3.  3.  6.  7.  8.  2. 11. 11.  1.  3.], Labels=[1 1 0]
Item#1: Features=[7. 6. 4. 4. 6. 8. 3. 4. 6. 4.], Labels=[0 0 0]
Item#2: Features=[ 5.  5. 13.  7.  6.  3.  6. 11.  4.  2.], Labels=[1 1 0]
Item#3: Features=[1. 1. 5. 5. 7. 3. 4. 6. 4. 4.], Labels=[1 1 1]
Item#4: Features=[ 4.  2.  3. 13.  7.  2.  4. 12.  1.  7.], Labels=[0 1 0]


**3. Creating the Neural Network**

In [24]:
# Defining the model: Multilayer Perceptron (MLP)
model = Sequential()

# Number of nodes in the output layer = the number of labels
# Note: The hidden layer has 20 nodes that were chosen by trial and error
model.add(Dense(20, input_dim = numberOfFeatures, kernel_initializer='he_uniform', activation='relu'))
# Output layer to estimate the membership probability
# Note: Sigmoid activation (0.00 to 1.00) is used for each node in the output layer 
model.add(Dense(numberOfClasses, activation='sigmoid'))
# Binary cross-entropy loss function + Adam version of GDA
model.compile(loss='binary_crossentropy', optimizer='adam')

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 20)                220       
                                                                 
 dense_3 (Dense)             (None, 3)                 63        
                                                                 
Total params: 283
Trainable params: 283
Non-trainable params: 0
_________________________________________________________________


In [25]:
results = list()
numberOfFolds = 10 # Folds in Cross Validation
repeats = 3 # Repeats in Cross Validation
verbose = 0 # Verbose in fitting the model
epochs = 50 # Number of epochs

inputsCount, outputsCount = features.shape[1], labels.shape[1]
print(f'The # of inputs and outputs are {inputsCount} and {outputsCount}, respectively.')

# Evaluation procedure
# Using repeated k-fold cross-validation instead of train/test split
# Note: It gets an unbiased estimate of model performance
crossValidation = RepeatedKFold(n_splits = numberOfFolds, n_repeats = repeats, 
                   random_state = randomState)

# Iterate over folds
index = 0
for trainItem, testItem in crossValidation.split(features):
  # Data Preparation
  featTrain, featTest = features[trainItem], features[testItem]
  classTrain, classTest = labels[trainItem], labels[testItem]
  # Fitting the model
  model.fit(featTrain, classTrain, verbose = verbose, epochs = epochs)
  # Making a prediction on the test set
  # Note: It predicts three probabilities for each sample
  predictedClass = model.predict(featTest).round(3)
  # Calculating the accuracy
  accuracy = accuracy_score(classTest, predictedClass.round())
  index += 1
  print(f'[{index}] Predicted: {predictedClass[0]}, Accuracy: {accuracy}')
  results.append(accuracy)

# Final Accuracy
print(f'Mean Accuracy: {round(mean(results), 3)}, Standard Deviation: {round(std(results), 3)}')

The # of inputs and outputs are 10 and 3, respectively.
[1] Predicted: [0.043 0.997 0.008], Accuracy: 0.56
[2] Predicted: [0.297 0.746 0.472], Accuracy: 0.8
[3] Predicted: [0.532 0.017 0.78 ], Accuracy: 0.74
[4] Predicted: [0.001 0.088 0.   ], Accuracy: 0.78
[5] Predicted: [0.998 0.954 0.004], Accuracy: 0.8
[6] Predicted: [0.982 0.995 0.874], Accuracy: 0.86
[7] Predicted: [0.003 0.894 0.   ], Accuracy: 0.9
[8] Predicted: [0.968 0.983 0.371], Accuracy: 0.88
[9] Predicted: [0.93  0.999 0.969], Accuracy: 0.88
[10] Predicted: [0.005 0.    0.   ], Accuracy: 0.82
[11] Predicted: [0.059 1.    0.   ], Accuracy: 0.84
[12] Predicted: [0.96  1.    0.986], Accuracy: 0.88
[13] Predicted: [0.949 0.973 0.118], Accuracy: 0.86
[14] Predicted: [1.    0.069 0.   ], Accuracy: 0.84
[15] Predicted: [0.    0.095 0.   ], Accuracy: 0.86
[16] Predicted: [0.059 1.    0.   ], Accuracy: 0.9
[17] Predicted: [1.    0.992 0.003], Accuracy: 0.9
[18] Predicted: [0.822 1.    0.   ], Accuracy: 0.88
[19] Predicted: [0.028

**4. Predicting a New Value**

In [31]:
# Make a prediction for new data
newSample = asarray([[7, 4, 6, 6, 4, 8, 3, 5, 11, 3]])

newPrediction = model.predict(newSample)
print(f'Prediction Result: {newPrediction[0].round(5)}')

Prediction Result: [4.e-05 0.e+00 0.e+00]
